# Ejercicio 3 - Trasponer matriz con coalescencia usando memoria compartida

Partiendo del código suministrado:

* Completar los fragmentos indicados con `TODO` 

* Comparar las prestaciones de ambos kernel

In [ ]:
# Ejecutar en Google Colab
!pip install numpy matplotlib scikit-image numba cython setuptools

### EVITAR ERRORES

!uv pip install -q --system numba-cuda==0.4.0

from numba import config
config.CUDA_ENABLE_PYNVJITLINK = 1

In [ ]:
from numba import cuda, types as numba_types
import numpy as np
n = 4096*4096 # 16M

@cuda.jit
def transpose(a, transposed):
    x, y = cuda.grid(2)
    transposed[x][y] = a[y][x]

@cuda.jit
def tile_transpose(a, transposed):
    # Suponer bloques 32x32

    # TODO 1: crear array 32x32 en memoria compartida

    # Calcular índices en array a
    a_row, a_col = cuda.grid(2)
   
    # TODO 2: leer de memoria global y escribir en memoria compartida
    #   los índices locales serán para el buffer en memoria compartida
    #   y los globales para el array a
    
    # TODO 3: Esperamos a que todos los hilos del bloque actualicen la escritura
 
    t_row, t_col = cuda.grid(2)
    # TODO 4: Escribir de memoria compartida (usando índices de hilo)
    #   a memoria global (usando índices de grid) transponiendo cada elemento
    
threads_per_block = (32, 32) # 2D blocks
blocks = (128, 128) #2D grid

# 4096x4096 input and output matrices
h_a = np.arange(n).reshape((4096,4096)).astype(np.float32)
d_a = cuda.to_device(h_a)
d_transposed = cuda.device_array(shape=(4096,4096), dtype=np.float32)

# Invocación a traspose y comprobación
transpose[blocks, threads_per_block](d_a, d_transposed)
result = d_transposed.copy_to_host()
expected = h_a.T
np.testing.assert_equal(result, expected)

# Invocación a tile_traspose y comprobación
tile_transpose[blocks, threads_per_block](d_a, d_transposed)
result = d_transposed.copy_to_host()
expected = h_a.T
np.testing.assert_equal(result, expected)

### Sergio

In [ ]:
from numba import cuda, types as numba_types
import numpy as np
n = 4096*4096 # 16M

@cuda.jit
def transpose(a, transposed):
    x, y = cuda.grid(2)
    transposed[x][y] = a[y][x]

@cuda.jit
def tile_transpose(a, transposed):
    # Suponer bloques 32x32

    # TODO 1: crear array 32x32 en memoria compartida
    temp = cuda.shared.array(shape=(32, 32), dtype = numba_types.float32)

    # Calcular índices en array a
    #   los incrementos de threadIdx.x deben corresponderse con las columnas de a
    a_col = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    a_row = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y
   
    # TODO 2: leer de memoria global y escribir en memoria compartida
    #   los índices locales serán para el buffer en memoria compartida
    #   y los globales para el array a
    temp[cuda.threadIdx.x][cuda.threadIdx.y] = a[a_row][a_col]
    # TODO 3: Esperamos a que todos los hilos del bloque actualicen la escritura
    cuda.syncthreads()
    # Calcular índices en array a
    #  los incrementos de threadIdx.x deben corresponderse a las columnas pero escribimos
    #  en posiciones traspuestas del grid
    t_col = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.x
    t_row = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.y
    
    # TODO 4: Escribir de memoria compartida (usando índices de hilo)
    #   a memoria global (usando índices de grid) transponiendo cada elemento
    x, y = cuda.grid(2)
    transposed[x][y] = temp[t_row][t_col]

threads_per_block = (32, 32) # 2D blocks
blocks = (128, 128) #2D grid

# 4096x4096 input and output matrices
h_a = np.arange(n).reshape((4096,4096)).astype(np.float32)

d_a = cuda.to_device(h_a)
d_transposed = cuda.device_array(shape=(4096,4096), dtype=np.float32)

# Invocación a traspose y comprobación
transpose[blocks, threads_per_block](d_a, d_transposed)
result = d_transposed.copy_to_host()
expected = h_a.T
np.testing.assert_equal(result, expected)

# Invocación a tile_traspose y comprobación
tile_transpose[blocks, threads_per_block](d_a, d_transposed)
result = d_transposed.copy_to_host()
expected = h_a.T
np.testing.assert_equal(result, expected)

In [ ]:
from numba import cuda, types as numba_types
import numpy as np
n = 4096*4096 # 16M

@cuda.jit
def transpose(a, transposed):
    x, y = cuda.grid(2)
    transposed[x][y] = a[y][x]

@cuda.jit
def tile_transpose(a, transposed):
    # Suponer bloques 32x32

    # TODO 1: crear array 32x32 en memoria compartida
    temp = cuda.shared.array(shape=(32, 32), dtype = numba_types.float32)

    # Calcular índices en array a
    #   los incrementos de threadIdx.x deben corresponderse con las columnas de a
    x, y = cuda.grid(2)

    # TODO 2: leer de memoria global y escribir en memoria compartida
    #   los índices locales serán para el buffer en memoria compartida
    #   y los globales para el array a
    temp[cuda.threadIdx.y][cuda.threadIdx.x] = a[y][x] #los warp se asocian por columna
    #debemos hacer que una columna lea una fila de a.
    #como está ahora se transponen los cuatro bloques
    # TODO 3: Esperamos a que todos los hilos del bloque actualicen la escritura
    cuda.syncthreads()
    # Calcular índices en array a
    #  los incrementos de threadIdx.x deben corresponderse a las columnas pero escribimos
    #  en posiciones traspuestas del grid
    t_row = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.y
    t_col = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.x
    #con esto lo que hacemos es transponer dentro del bloque

    # TODO 4: Escribir de memoria compartida (usando índices de hilo)
    #   a memoria global (usando índices de grid) transponiendo cada elemento
    transposed[t_row][t_col] = temp[cuda.threadIdx.x][cuda.threadIdx.y]

threads_per_block = (32, 32) # 2D blocks
blocks = (128, 128) #2D grid

# 4096x4096 input and output matrices
h_a = np.arange(n).reshape((4096,4096)).astype(np.float32)

d_a = cuda.to_device(h_a)
d_transposed = cuda.device_array(shape=(4096,4096), dtype=np.float32)

# Invocación a traspose y comprobación
transpose[blocks, threads_per_block](d_a, d_transposed)
result = d_transposed.copy_to_host()
expected = h_a.T
np.testing.assert_equal(result, expected)

# Invocación a tile_traspose y comprobación
tile_transpose[blocks, threads_per_block](d_a, d_transposed)
result = d_transposed.copy_to_host()
expected = h_a.T
np.testing.assert_equal(result, expected)

In [ ]:
%timeit transpose[blocks, threads_per_block](d_a, d_transposed); cuda.synchronize()
%timeit tile_transpose[blocks, threads_per_block](d_a, d_transposed); cuda.synchronize()